In [20]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
from datetime import datetime

import requests
import json

In [47]:
# keep columns
keep_cols = [
    'id',
    'utcDate',
    'status',
    'matchday',
    # 'season.id',
    # 'season.currentMatchday',
    # 'homeTeam.id',
    # 'homeTeam.name',
    'homeTeam.shortName',
    # 'awayTeam.id',
    # 'awayTeam.name',
    'awayTeam.shortName',
    # 'score.winner',
    'score.fullTime.home',
    'score.fullTime.away',
    # 'score.halfTime.home',
    # 'score.halfTime.away'
]

In [67]:
uri = 'http://api.football-data.org/v4/competitions/PL/matches?status=FINISHED'
headers = { 'X-Auth-Token': '73015e4f2665414cb1fafe97e8f9971e' }

response = requests.get(uri, headers=headers)
data = response.json()
df = pd.json_normalize(data, record_path = ['matches'])
df = df[keep_cols]
df['utcDate'] = pd.to_datetime(df['utcDate'])
df['score.fullTime'] = df['score.fullTime.home'].astype(str) + '-' + df['score.fullTime.away'].astype(str)
print(df.shape)
df.head()


(126, 9)


,id,utcDate,status,matchday,homeTeam.shortName,awayTeam.shortName,score.fullTime.home,score.fullTime.away,score.fullTime
0,416384,2022-08-05 19:00:00+00:00,FINISHED,1,Crystal Palace,Arsenal,0,2,0-2
1,416383,2022-08-06 11:30:00+00:00,FINISHED,1,Fulham,Liverpool,2,2,2-2
2,416378,2022-08-06 14:00:00+00:00,FINISHED,1,Tottenham,Southampton,4,1,4-1
3,416379,2022-08-06 14:00:00+00:00,FINISHED,1,Newcastle,Nottingham,2,0,2-0
4,416381,2022-08-06 14:00:00+00:00,FINISHED,1,Leeds United,Wolverhampton,2,1,2-1


In [68]:
# pivot
df_table = df.pivot_table(index=['homeTeam.shortName'], columns=['awayTeam.shortName'], fill_value='.', values=['score.fullTime'], aggfunc='first')

df_table

score.fullTime                                    \
awayTeam.shortName        Arsenal Aston Villa Bournemouth Brentford   
homeTeam.shortName                                                    
Arsenal                         .         2-1           .         .   
Aston Villa                     .           .           .       4-0   
Bournemouth                   0-3         2-0           .       0-0   
Brentford                     0-3           .           .         .   
Brighton Hove                   .           .           .         .   
Chelsea                         .           .           .         .   
Crystal Palace                0-2         3-1           .       1-1   
Everton                         .           .           .         .   
Fulham                          .         3-0         2-2       3-2   
Leeds United                  0-1         0-0           .         .   
Leicester City                  .           .           .       2-2   
Liverpool                       .           .         9-0         .   
Man City                        .           .         4-0         .   
Man United                    3-1           .           .         .   
Newcastle                       .         4-0         1-1       5-1   
Nottingham                      .         1-1         2-3         .   
Southampton                   1-1           .           .         .   
Tottenham                       .           .           .         .   
West Ham                        .           .         2-0         .   
Wolverhampton                   .           .           .         .   

                                                                        \
awayTeam.shortName Brighton Hove Chelsea Crystal Palace Everton Fulham   
homeTeam.shortName                                                       
Arsenal                        .       .              .       .    2-1   
Aston Villa                    .     0-2              .     2-1      .   
Bournemouth                    .       .              .       .      .   
Brentford                    2-0     0-0              .     1-1      .   
Brighton Hove                  .     4-1              .       .      .   
Chelsea                        .       .              .       .      .   
Crystal Palace                 .     1-2              .       .      .   
Everton                        .     0-1            3-0       .      .   
Fulham                       2-1       .              .     0-0      .   
Leeds United                   .     3-0              .     1-1    2-3   
Leicester City                 .       .            0-0       .      .   
Liverpool                    3-3       .            1-1       .      .   
Man City                     3-1       .            4-2       .      .   
Man United                   1-2       .              .       .      .   
Newcastle                      .       .            0-0     1-0      .   
Nottingham                     .       .              .       .    2-3   
Southampton                    .     2-1              .     1-2      .   
Tottenham                      .       .              .     2-0    2-1   
West Ham                     0-2       .              .       .    3-1   
Wolverhampton                  .       .              .       .    0-0   

                                                                              \
awayTeam.shortName Leeds United Leicester City Liverpool Man City Man United   
homeTeam.shortName                                                             
Arsenal                       .            4-2       3-2        .          .   
Aston Villa                   .              .         .      1-1          .   
Bournemouth                   .            2-1         .        .          .   
Brentford                   5-2              .         .        .        4-0   
Brighton Hove               1-0            5-2         .        .          .   
Chelsea                       .           

## Create Matches DataFrame with a row for each team 

In [65]:
points_map = {
    'W': 3,
    'D': 1,
    'L': 0
}

def get_result(score, score_opp):
    if score == score_opp:
        return 'D'
    elif score > score_opp:
        return 'W'
    else:
        return 'L'

drop_cols = ['homeTeam.shortName','awayTeam.shortName', 'score.fullTime.home', 'score.fullTime.away']

# convert each match into two rows (one for each team)

df['H'] = df['homeTeam.shortName']
df['A'] = df['awayTeam.shortName']

df_matches = pd.melt(
    df,
    id_vars=keep_cols,
    value_vars=["H", "A"],
    var_name="home_away",
    value_name="team",
)
df_matches['opponent'] = np.where(
    df_matches['home_away'] == 'H', 
    df_matches['awayTeam.shortName'], 
    df_matches['homeTeam.shortName']
    )

# full time goals
df_matches["goals_scored"] = np.where(
    df_matches["team"] == df_matches["homeTeam.shortName"],
    df_matches["score.fullTime.home"],
    df_matches["score.fullTime.away"],
)
df_matches["goals_against"] = np.where(
    df_matches["team"] != df_matches["homeTeam.shortName"],
    df_matches["score.fullTime.home"],
    df_matches["score.fullTime.away"],
)
df_matches["result"] = np.vectorize(get_result)(
    df_matches["goals_scored"], df_matches["goals_against"]
)
df_matches["points"] = df_matches["result"].map(points_map)


# sort df_matches by id
df_matches = (df_matches
    .drop(drop_cols, axis=1)
    .sort_values(by=['utcDate', 'id'], ascending=True)
    .reset_index(drop=True)
    )
df_matches.head(20)

,id,utcDate,status,matchday,home_away,team,opponent,goals_scored,goals_against,result,points
0,416384,2022-08-05 19:00:00+00:00,FINISHED,1,H,Crystal Palace,Arsenal,0,2,L,0
1,416384,2022-08-05 19:00:00+00:00,FINISHED,1,A,Arsenal,Crystal Palace,2,0,W,3
2,416383,2022-08-06 11:30:00+00:00,FINISHED,1,H,Fulham,Liverpool,2,2,D,1
3,416383,2022-08-06 11:30:00+00:00,FINISHED,1,A,Liverpool,Fulham,2,2,D,1
4,416378,2022-08-06 14:00:00+00:00,FINISHED,1,H,Tottenham,Southampton,4,1,W,3
5,416378,2022-08-06 14:00:00+00:00,FINISHED,1,A,Southampton,Tottenham,1,4,L,0
6,416379,2022-08-06 14:00:00+00:00,FINISHED,1,H,Newcastle,Nottingham,2,0,W,3
7,416379,2022-08-06 14:00:00+00:00,FINISHED,1,A,Nottingham,Newcastle,0,2,L,0
8,416381,2022-08-06 14:00:00+00:00,FINISHED,1,H,Leeds United,Wolverhampton,2,1,W,3
9,416381,2022-08-06 14:00:00+00:00,FINISHED,1,A,Wolverhampton,Leeds United,1,2,L,0


score                                                  \
opponent       Arsenal Aston Villa Bournemouth Brentford Brighton Hove   
team                                                                     
Arsenal              .         2-1         0-3       0-3             .   
Aston Villa        2-1           .         2-0       4-0             .   
Bournemouth        0-3         2-0           .       0-0             .   
Brentford          0-3         4-0         0-0         .           2-0   
Brighton Hove        .           .           .       2-0             .   
Chelsea              .         0-2           .       0-0           4-1   
Crystal Palace     0-2         3-1           .       1-1             .   
Everton              .         2-1           .       1-1             .   
Fulham             2-1         3-0         2-2       3-2           2-1   
Leeds United       0-1         0-0           .       5-2           1-0   
Leicester City     4-2           .         2-1       2-2           5-2   
Liverpool          3-2           .         9-0         .           3-3   
Man City             .         1-1         4-0         .           3-1   
Man United         3-1           .           .       4-0           1-2   
Newcastle            .         4-0         1-1       5-1           0-0   
Nottingham         5-0         1-1         2-3         .           0-0   
Southampton        1-1         1-0         0-1         .             .   
Tottenham          3-1           .         2-3         .           0-1   
West Ham             .         0-1         2-0         .           0-2   
Wolverhampton        .           .         0-0       1-1             .   

                                                                   \
opponent       Chelsea Crystal Palace Everton Fulham Leeds United   
team                                                                
Arsenal              .            0-2       .    2-1          0-1   
Aston Villa        0-2            3-1     2-1    3-0          0-0   
Bournemouth          .              .       .    2-2            .   
Brentford          0-0            1-1     1-1    3-2          5-2   
Brighton Hove      4-1              .       .    2-1          1-0   
Chelsea              .            1-2     0-1      .          3-0   
Crystal Palace     1-2              .     3-0      .          2-1   
Everton            0-1            3-0       .    0-0          1-1   
Fulham               .              .     0-0      .          2-3   
Leeds United       3-0            2-1     1-1    2-3            .   
Leicester City     2-1            0-0       .      .          2-0   
Liverpool            .            1-1     0-0    2-2          1-2   
Man City             .            4-2       .      .            .   
Man United         1-1              .     1-2      .            .   
Newcastle            .            0-0     1-0    1-4            .   
Nottingham           .              .     1-1    2-3            .   
Southampton        2-1            1-0     1-2      .          2-2   
Tottenham          2-2              .     2-0    2-1            .   
West Ham           2-1              .     1-0    3-1            .   
Wolverhampton      3-0            2-1       .    0-0          2-1   

                                                                       \
opponent       Leicester City Liverpool Man City Man United Newcastle   
team                                                                    
Arsenal                   4-2       3-2        .        3-1         .   
Aston Villa                 .         .      1-1          .       4-0   
Bournemouth               2-1       9-0      4-0          .       1-1   
Brentford                 2-2         .        .        4-0       5-1   
Brighton Hove             5-2       3-3      3-1        1-2       0-0   
Chelsea                   2-1         .        .        1-1         .   
Crystal Palace            0-0       1-1      4-2          .       0-0